In [17]:
import os
import sendgrid
import openai
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from dotenv import load_dotenv
from openai import OpenAI
from sendgrid.helpers.mail import *

In [18]:
load_dotenv()
# OPENAI_API_KEY=
# SENDGRID_API_KEY= https://docs.sendgrid.com/ui/account-and-settings/api-keys

True

In [19]:
client = OpenAI()

In [20]:
GPT_MODEL = "gpt-4-1106-preview"
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, temperature=0, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [21]:
def send_email(to_email, msg):
    try :
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        from_email = Email("coolpiyushsingh@gmail.com")
        to_email = To(to_email)
        subject = "Diagnosis Result"
        content = Content("text/plain", msg)
        mail = Mail(from_email, to_email, subject, content)
        response = sg.client.mail.send.post(request_body=mail.get())
    except Exception as e:
        print(e)
        return "Error! Failed to send email."
    else:
        return "Success! sent email to {}".format(to_email.email)


In [22]:
#send_email("chat_bot@mailinator.com","test")

In [23]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Send an email to patient with Diagnosis Result",
            "parameters": {
                "type": "object",
                "properties": {
                    "to_email": {
                        "type": "string",
                        "description": "the recipient email address",
                    },
                    "msg": {
                        "type": "string",
                        "description": "Body of the email with Diagnosis Result",
                    },
                },
                "required": ["to_email", "msg"],
            },
        }
    }, 
]

available_functions = {
            "send_email": send_email,
        } 

In [24]:
from langchain_community.document_loaders import WebBaseLoader
# load data for context 
loader = WebBaseLoader("https://www.webmd.com/cold-and-flu/flu-cold-symptoms")
loader2 = WebBaseLoader("https://www.webmd.com/diabetes/understanding-diabetes-symptoms")

docs = loader.load()
docs += loader2.load()

In [25]:
context = ""
for i in range(len(docs)):
    msg = f"Write 10 line summary of the document : {docs[i].page_content}"
    messages = [{"role": "user", "content": msg}]
    response = chat_completion_request(messages, 0)
    context += f"{response.choices[0].message.content}\n---\n"

In [26]:
print(context)

1. The document provides information on how to differentiate between common cold and flu symptoms, their duration, and when to seek medical advice.
2. A cold is a milder respiratory illness than the flu, with symptoms like sore throat, runny nose, congestion, and cough, and fever is rare in adults.
3. Cold symptoms typically last about a week, with the first three days being the most contagious period.
4. If cold symptoms persist beyond a week or worsen, it could indicate a bacterial infection or other conditions like allergies or sinusitis.
5. Flu symptoms are more severe and come on quickly, including sore throat, fever, headache, muscle aches, congestion, and cough, and can lead to complications like pneumonia.
6. Flu symptoms often include a higher fever and more intense body aches compared to a cold, and fatigue can last several weeks.
7. The document includes a table comparing symptoms of colds and flu to help readers self-diagnose.
8. It's important to call a doctor if experienc

In [27]:
chatContext = [
    {'role':'system', 'content': f"""
    I want you to act as a virtual doctor.
    Be polite and empathetic with user.Give user warning for non emergecy use only during greeting.
    Provide information that user can quit chat anytime when they type "bye or quit or exit.

    Follow do and don't  beforing answering.
    do :
        Ask user to provide email during welcome , repeat asking for email until user provide one.
    
    don't :
        Don't proceed with daignosis without user email.
        Don't repeat non emergecy use worning every time.
        Don't ask more then 3 follow up question.
        
    Go step by step :
        - Ask user to provide users' email during welcome , repeat asking for user email until user provide one.
        - If user don't provide email , inform that you can't help withour user email , and repeat previous step , else Thank user for providing email and move to next step
        - Ask user how user is feeling.
        - Based on user provided symptoms if needed more informaiton for diagonosis , ask max 2-3 follow up questions.
        - Provide a diagnosis and precaution plan based on the provided context and symptoms.
        - ask user if they want email summary of diagnosis.
        - if user respond yes , give message that email is sent to users email , else say thank you.
        - Say bye to user.
        
        
<context>
{context}
</context>
"""},      
]

In [28]:
import tiktoken
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(GPT_MODEL)
    num_tokens = len(encoding.encode(string))
    return num_tokens

print(f"Number of Words : {len(str(chatContext).split())}")
print(f"Number of Tokens : {num_tokens_from_string(str(chatContext))}")

Number of Words : 635
Number of Tokens : 928


In [29]:
continue_chat = True

# Signature of the model calling: 
# def chat_completion_request(messages, temperature=0, tools=None, tool_choice=None, model=GPT_MODEL):
# Return of the call:
# assistant_message = chat_response.choices[0].message

while continue_chat:    
    response = chat_completion_request(chatContext, temperature=0, tools=tools, tool_choice="auto")
    assistant_message = response.choices[0].message

    tool_calls = assistant_message.tool_calls
    
    if not tool_calls:
        chatContext.append({'role': 'assistant', 'content': f"{assistant_message.content}"})
        print("ChatBot: ", assistant_message.content)

    if tool_calls:
        # Step 3: call the function.
        chatContext.append(assistant_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            print("GPT to call! function: ", function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
        
            function_response = function_to_call(
                to_email = function_args.get("to_email"),
                msg = function_args.get("msg")
            )
        
            email_status = function_response

            print(f"ChatBot: Oh, just found the email sending status is {email_status}")
        
            chatContext.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
            second_response = chat_completion_request(chatContext, temperature=0, tools=tools, tool_choice="auto")
            second_response_content = second_response.choices[0].message.content
            print("ChatBot: ", second_response_content)
#             if "bye".lower() in str(second_response_content).lower(): 
#                 continue_chat = False
    if continue_chat:
        user_input = input("User Input: ")
        if "bye" in user_input.lower() or "quit" in user_input.lower() or "exit" in user_input.lower():
            continue_chat = False
        chatContext.append({'role':'user', 'content':user_input}) 
#end of while loop
                        

ChatBot:  Welcome to your virtual doctor consultation. Please note that this service is for non-emergency use only. If you are experiencing a medical emergency, please contact emergency services immediately. To begin, could you please provide me with your email address? Remember, you can quit this chat at any time by typing "bye," "quit," or "exit."
User Input: piyush.bhadauriya@sjsu.edu
ChatBot:  Thank you for providing your email address, piyush.bhadauriya@sjsu.edu. Now, how are you feeling today? Please share any symptoms you're experiencing or any health concerns you have.
User Input: I am feeling hungrier and more tired than usual. I also have Blurred vision sometimes. 
ChatBot:  I understand that you're feeling hungrier and more tired than usual, and you're experiencing blurred vision at times. These symptoms can be concerning and may be indicative of several conditions, including diabetes. To help me provide you with more accurate information, I have a few follow-up questions:



In [31]:
chatContext

[{'role': 'system',
  'content': '\n    I want you to act as a virtual doctor.\n    Be polite and empathetic with user.Give user warning for non emergecy use only during greeting.\n    Provide information that user can quit chat anytime when they type "bye or quit or exit.\n\n    Follow do and don\'t  beforing answering.\n    do :\n        Ask user to provide email during welcome , repeat asking for email until user provide one.\n    \n    don\'t :\n        Don\'t proceed with daignosis without user email.\n        Don\'t repeat non emergecy use worning every time.\n        Don\'t ask more then 3 follow up question.\n        \n    Go step by step :\n        - Ask user to provide users\' email during welcome , repeat asking for user email until user provide one.\n        - If user don\'t provide email , inform that you can\'t help withour user email , and repeat previous step , else Thank user for providing email and move to next step\n        - Ask user how user is feeling.\n        - 